In [ ]:
import numpy as np 
import scipy
import matplotlib.pyplot as plt 
import pandas as pd
from numpy.linalg import inv
from scipy import fft

#Question 1
def least_squares(A,b):
    '''
    Solve the least_squares equation
    '''
    ans = inv(A.T@A)@(A.T) @ b 
    return ans 

def reg_least_squares(x,A,b,sigma):
    '''
    Solve the regularized least_squares equation 
    '''
    #I will skip a couple steps and use the solution from the first midterm to solve directly 
    ans = inv(A.T@A + sigma*np.identity(len(x)))@(A.T)@b
    return ans

def plot(x,y,f):
    '''
    plot our findings (x is the vector for years, y is the vector for the flow rate, f is for plotting the actual values)
    '''
    plt.figure(figsize = (7,7))
    for i in range(0,len(y)):
        plt.scatter(x[i],f[i],s = 5, c = 'g')
    for i in range(0,len(y)):
         plt.scatter(x[i],y[i],s = 5,c = 'b')
    plt.plot(x,y,c = 'r')
    plt.plot(x,f,c = 'y')
    plt.xlabel('Years')
    plt.ylabel('Flow Rate') 
    plt.title("Flow Rate Versus Year")
    plt.show()

def plot_individual(x,y): 
    '''
    plot our findings (x is the vector for years, y is the vector for the flow rate, f is for plotting the actual values)
    '''
    plt.figure(figsize = (7,7))
    for i in range(0,len(y)):
         plt.scatter(x[i],y[i],s = 5,c = 'b')
    #plt.plot(x,y,c = 'r')
    plt.xlabel('Years')
    plt.ylabel('Flow Rate') 
    plt.title("Flow Rate Versus Year")
    plt.show()

In [ ]:
#For grader: Please use this variable to test my code. 
#Enter a value between 1 and 45
#This variable will omit that number of entries in my matrix.  
omitted = 10

#read in our files 
data = pd.read_csv('nile.csv')

#convert pandas datafile to numpy 
data = pd.DataFrame(data).to_numpy()

#define the length of our data 
n = 100

time = []
for i in range(0,n):
    time.append(data[i][0])

f = []
for i in range(0,n):
    f.append(data[i][1])
    
f_temp = f.copy()
#get rid of some values 
for i in range(0,omitted):
    change = 0
    f_temp[i+45] = change
    
f_new = [] 
for i in range(0,len(f_temp)):
    if (f_temp[i] != 0):
        f_new.append(f_temp[i])

#define X
X = scipy.fft.dct(np.eye(len(data)), norm='ortho')
alpha = X.T @ f 

#get rid of the 0 entries and define as f_new and X_new 
range_new = []
for i in range(0,len(f_temp)):
    if (f_temp[i] == 0):
        range_new.append(i)

#manual delete, can be improved 
X_temp = X.copy()
res = range_new[::len(range_new)-1]
rg = [] 
for i in range(res[0],res[1]+1):
    rg.append(i)

X_temp = np.delete(X, rg, axis = 0)
X_new = X_temp.copy()

#run our algorithm to obtain our answer
alpha_least_squares = least_squares(X_new,f_new)
alpha_reg_least_squares = reg_least_squares(X,X_new,f_new,1) #sigma = 1
alpha_reg_least_squares_2 = reg_least_squares(X,X_new,f_new,10) #sigma = 10

f_least_squares = X @ alpha_least_squares
f_reg_least_squares = X @ alpha_reg_least_squares
f_reg_least_squares_2 = X @ alpha_reg_least_squares_2

print("Graph of Least Squares")
plot(time,f_least_squares,f)
print("Graph of Regularized Least Squares with Sigma = 1")
plot(time,f_reg_least_squares,f)
print("Graph of Least Squares with Sigma = 10")
plot(time,f_reg_least_squares_2,f)

In [ ]:
#Question 2 
'''
variables of optimization is ai = ui + vi for all i = 1,...,n
'''
from scipy.optimize import linprog 

# print(alpha) 
# print(X_new)
# print(f_new) 

#This should be XI 
A = X_new 
#should be fI
b = f_new

#define n 
n = len(A[0])

#define c
c = []
for i in range(0,n*2):
    c.append(1)

#define our negative A matrix
A_neg = np.negative(A) 

#create A_new matrix for the left side of our constraints 
A_new = []
for i in range(0,len(A)):
    temp = [] 
    for j in range(0,len(A[0])):
        temp.append(A[i][j])
    for k in range(0,len(A[0])):
        temp.append(A_neg[i][k])
    A_new.append(temp)

#set our bounds 
uv_bound = (0,None)
all_bounds = [] 
for i in range(0,len(c)):
    all_bounds.append(uv_bound)

#perform linear programming 
res = linprog(c, A_eq=A_new, b_eq=b, bounds=all_bounds)
print(res.message) #check if successful 

#correct our values since we want to convert back to min(x) by using abs(xi) = abs(ui) + abs(vi) [abs will not make a difference but I'm still including it to be safe]
true_ans = [] 
for i in range(0,n):
    u_plus_v = abs(res.x[i]) + abs(res.x[i+99])
    true_ans.append(u_plus_v)

#print out our answer
print(true_ans)

In [ ]:
#plot to compare the result in problem 2 to problem 1
f_scipy = X @ true_ans

print("Graph of Least Squares")
plot(time,f_least_squares,f)
print("Graph of Regularized Least Squares with Sigma = 1")
plot(time,f_reg_least_squares,f)
print("Graph of scipy.optimize.linprog")
plot(time, f_scipy,f)

In [ ]:
#Question 4

#define our n as length of f (since there are that many lambda corresponding to our f)
n = len(X_new[0]) #100
m = len(f_new) #90

#This should be XI (define our negative A matrix)

A = X_new.copy()
A_one = A.T
A_neg = np.negative(A_one) 

#define our c
c_new = f_new
c_neg = np.negative(f_new)  

#define our b 
one = []
for i in range(0,n*2):
    one.append(1)    
b = one 

#Build the A matrix (the big one!)
#n = 100, m = 90
A_new = []
for i in range(0,n):
    temp = [] 
    for j in range(0,m):
        temp.append(A_one[i][j])
    A_new.append(temp) 

for k in range(0,n):
    temp = []
    for l in range(0,m):
        temp.append(A_neg[k][l])
    A_new.append(temp)
    
#define our bound for all our lambda 
lambda_bound = (None,None)
tot_bound = []
for i in range(0,m):
    tot_bound.append(lambda_bound)
    
# print(c)
# print(A)
# #print(A_new)
# print(b)
# print(tot_bound)
res = linprog(c_neg, A_ub=A_new, b_ub=b, bounds=tot_bound)
print(res.message) #check if successful 
opt_lambda = res.x
print(res.x)

In [ ]:
#Comparing the results of the dual problem 
c_new = np.array(c_new)

left = c_new.T @ opt_lambda
print(left)

######
sum = 0 
for i in range(0,len(true_ans)):
    sum += true_ans[i]
    
print(sum)

#######
recovery = [] 

for j in range(0,len(c_new)):
    value = c_new[j] * opt_lambda[j]
    recovery.append(value)
    
recovery = np.array(recovery) 
plot_individual(time,recovery)

an = X_new.T @ recovery

plot(time,an,f)

In [ ]:
#bonus question 
import scipy.misc

img = (scipy.misc.face()/255).mean(2)
plt.gray()
plt.imshow(img)
plt.show()

n = 200
m = 400
p = 100

dct = lambda n: scipy.fft.dct(np.eye(n), norm='ortho')
X = np.kron(dct(m + 2*p), dct(n + 2*p))

# generate some random low-frequency DCT coefficients in 1D
alpha = np.zeros((m + 2*p)*(n + 2*p))
alpha[alpha.size//8] = np.random.randn(alpha.size//8)
# compute the linear combination---this is still 1D
f = X@alpha
# reshape the values of f into a 2D array
F = f.reshape(m + 2*p, n + 2*p)
# plot the values
plt.figure()
plt.imshow(F)
plt.show()